 # Train Object Detection Model on Google Colab

## Download Object Detection API

In [ ]:
!git clone --quiet https://github.com/tensorflow/models.git

## Install dependency

In [ ]:
!apt-get install -qq protobuf-compiler python-tk

!pip install -q Cython contextlib2 pillow lxml matplotlib PyDrive

!pip install -q pycocotools


## Run some setups and builds

In [ ]:
# Get into 'research/' folder
%cd models/research/

In [ ]:
# Run setups and builds
!protoc object_detection/protos/*.proto --python_out=.

!python setup.py build

!python setup.py install

In [ ]:
# Get into 'slim/' folder
%cd slim/

In [ ]:
# Run setups and builds
!python setup.py build

!python setup.py install

In [ ]:
# Add 'research/' and 'slim/' to Python path
import os
import sys
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'
sys.path.append("/content/models/research/slim/")

## Run test to check everything works fine

In [ ]:
%cd ..

!python object_detection/builders/model_builder_test.py

## Download a pre-trained model

In [ ]:
# Get into 'object_detection/' folder
%cd object_detection/

In [ ]:
# Download a pre-trained model
# change the model name by the model you want to use (exp SSD_mobilnet, ...)
import os
import shutil
import glob
import urllib.request
import tarfile
# Link of the models you can download:
# https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md
# MODEL = 'faster_rcnn_inception_v2_coco_2018_01_28'
MODEL = 'ssd_mobilenet_v2_coco_2018_03_29'
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = 'faster_rcnn_inception_v2_coco_2018_01_28'

if not (os.path.exists(MODEL_FILE)):
    #opener = urllib.request.urlretrieve()
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
#if (os.path.exists(DEST_DIR)):
#  shutil.rmtree(DEST_DIR)
#os.rename(MODEL, DEST_DIR)

## Prepare the configs

In [ ]:
# Create folders where to put the config files and to save the trained model
# In the new version of the Object Detection API, these folders are already created
%mkdir training train

In [ ]:
# Get into 'training/' folder
%cd training

#upload the config file from your PC

from google.colab import files
from os import path
uploaded = files.upload()
for name, data in uploaded.items():
    with open(name, 'wb') as f:
        f.write(data)
        f.close()
        print('saved file ' + name)

In [ ]:
# Copy the training dataset from your Drive to Google Colabs
# Make sure to put the train.record or train.tfrecord in the same path you specified
# in the config file
# Change 'train.record' according to your settings

%cd ..

from google.colab import drive
drive.mount('/content/drive')

# !ls "/content/drive/My Drive"

!cp "/content/drive/My Drive/train.record" "train.record"

## Train the model: Plase make sure of the paths, I may got confused

In [ ]:
# copy the train.py script from 'legacy/' folder
%cd ../legacy
%cp train.py ../

In [ ]:
# Run the training
%cd ..
# Change 'ssd_mobilenet_v1_pets.config' by the config file you used
# 'train/': the folder where to save your model
!python train.py --logtostderr --train_dir=train/ --pipeline_config_path=training/ssd_mobilenet_v1_pets.config

## Export the model to an inference graph:

In [ ]:
# Change 18000 with the last checkpoint your model reached.

%mkdir ouput
!python export_inference_graph.py --input_type image_tensor --pipeline_config_path training/ssd_mobilenet_v1_pets.config --trained_checkpoint_prefix train/model.ckpt-18000 --output_directory output/

## Test the created inference graph:

In [ ]:
# Upload some images to test the model  

%mkdir test_imgs/
%cd test_imgs/
# You can upload multiple files at the same time
from google.colab import files
from os import path
uploaded = files.upload()
for name, data in uploaded.items():
    with open(name, 'wb') as f:
        f.write(data)
        f.close()
        print('saved file ' + name)

In [ ]:
# Test the model 
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import cv2
import glob

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops
print(tf.__version__)
if tf.__version__ < '1.04.0':
    raise ImportError('Please upgrade your tensorflow installation to v1.4.* or later!')

# This is needed to display the images.
%matplotlib inline


from utils import label_map_util
from utils import visualization_utils as vis_util


# What model to download.
# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = 'output' + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('training/', 'labelmap.pbtxt')

# Change the number according the how many classes you have
NUM_CLASSES = 1


detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)



def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)



# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
TEST_IMAGE_PATHS = glob.glob('test_imgs/*')
# Size, in inches, of the output images.
IMAGE_SIZE = (36, 24)




def run_inference_for_single_image(image, graph):
    with graph.as_default():
        with tf.Session() as sess:
            # Get handles to input and output tensors
            ops = tf.get_default_graph().get_operations()
            all_tensor_names = {output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in [
                'num_detections', 'detection_boxes', 'detection_scores',
                'detection_classes', 'detection_masks'
            ]:
                tensor_name = key + ':0'
                if tensor_name in all_tensor_names:
                    tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                        tensor_name)
            if 'detection_masks' in tensor_dict:
                # The following processing is only for single image
                detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
                detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
                # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
                real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
                detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
                detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
                detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                    detection_masks, detection_boxes, image.shape[0], image.shape[1])
                detection_masks_reframed = tf.cast(
                    tf.greater(detection_masks_reframed, 0.5), tf.uint8)
                # Follow the convention by adding back the batch dimension
                tensor_dict['detection_masks'] = tf.expand_dims(
                    detection_masks_reframed, 0)
            image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

            # Run inference
            output_dict = sess.run(tensor_dict,
                                   feed_dict={image_tensor: np.expand_dims(image, 0)})

            # all outputs are float32 numpy arrays, so convert types as appropriate
            output_dict['num_detections'] = int(output_dict['num_detections'][0])
            output_dict['detection_classes'] = output_dict[
                'detection_classes'][0].astype(np.uint8)
            output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
            output_dict['detection_scores'] = output_dict['detection_scores'][0]
            if 'detection_masks' in output_dict:
                output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict




for image_path in TEST_IMAGE_PATHS:
    image = Image.open(image_path)
    width, height = image.size
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = load_image_into_numpy_array(image)
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    # Actual detection.
    output_dict = run_inference_for_single_image(image_np, detection_graph)
    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        output_dict['detection_boxes'],
        output_dict['detection_classes'],
        output_dict['detection_scores'],
        category_index,
        instance_masks=output_dict.get('detection_masks'),
        use_normalized_coordinates=True,
        line_thickness=8)
  
    ymin = int((output_dict['detection_boxes'][0][0]*height))
    xmin = int((output_dict['detection_boxes'][0][1]*width))
    ymax = int((output_dict['detection_boxes'][0][2]*height))
    xmax = int((output_dict['detection_boxes'][0][3]*width))
    img_np = cv2.imread(image_path)
    Result = np.array(img_np[ymin:ymax,xmin:xmax])

    cv2.imwrite('cropped.jpg', Result)

    #print(output_dict['detection_boxes'][0])
    plt.figure(figsize=IMAGE_SIZE)
    plt.imshow(image_np)
    #plt.show()
  

## Compress the model into zip file

In [ ]:
# After testing the model and making sure it works well, we need to compress and put it
#in your Drive to use it later

import shutil
shutil.make_archive('output', 'zip', 'output')

In [ ]:
# Upload the model to your Drive


from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

uploaded = drive.CreateFile({'title': 'output.zip'})
uploaded.SetContentFile('output.zip')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))